In [ ]:
# === Cell 1: Unified Environment & Project-Wide Setup ===
import os, json, math, datetime as dt
from datetime import datetime  # for human-readable prints
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional (map preview): keep if you use geemap
try:
    import geemap
    GEEMAP_AVAILABLE = True
except Exception:
    GEEMAP_AVAILABLE = False

plt.style.use('seaborn-v0_8-whitegrid')
os.makedirs('outputs', exist_ok=True)

import ee

# --- Earth Engine authentication and initialization ---
# You can override with: export EE_PROJECT_ID=your-project-id
EE_PROJECT_ID = os.environ.get('EE_PROJECT_ID', 'nasa-flood')  # <— 원하는 기본 프로젝트 ID
def _ee_init():
    try:
        ee.Initialize()
        return "Initialized with default credentials"
    except Exception:
        try:
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Initialized with project='{EE_PROJECT_ID}'"
        except Exception:
            print("Authenticating with Earth Engine...")
            ee.Authenticate()
            ee.Initialize(project=EE_PROJECT_ID)
            return f"Authenticated & initialized with project='{EE_PROJECT_ID}'"

print(_ee_init())
print(f"Current time: {datetime.now().isoformat(timespec='seconds')}")

# ===== Project-wide constants =====
CFG = {
    # AOIs (EPSG:4326)
    # Mekong Delta (VN lower basin); box chosen for stability/reproducibility
    "AOI_DELTA": ee.Geometry.Rectangle([104.30,  8.50, 106.90, 10.90], geodesic=False),
    # Tonlé Sap (KH); "Mekong's heartbeat"
    "AOI_TONLESAP": ee.Geometry.Rectangle([103.30, 12.00, 105.20, 13.70], geodesic=False),

    # Analysis windows
    "YEARS": list(range(2015, 2025)),
    "FLOOD_MONTHS": (8, 9),     # Aug–Sep (wet-season peak)
    "DROUGHT_MONTHS": (3, 4),   # Mar–Apr (dry-season trough)

    # Thresholds (physical/empirical, fixed for comparability)
    "TH_VV_DB": -16.0,
    "TH_VH_DB": -22.0,

    # Landsat5 baseline window (pre-major-dam reference)
    "BASELINE_YEARS": [2005, 2006, 2007, 2008],

    # Event markers for plots
    "EVENTS": {
        "JINGHONG_FLOW_CUT": "2019-07-15",  # Jinghong flow cut (smoking gun)
        "XIAOWAN_ONLINE":    "2009-01-01",
        "NUOZHADU_ONLINE":   "2012-01-01"
    }
}

# ===== Utilities for Earth Engine =====
def _daterange_of_year_months(year:int, m1:int, m2:int):
    """Return ISO start and inclusive end-of-month last day for [m1..m2]."""
    start = dt.date(year, m1, 1)
    if m2 == 12:
        end = dt.date(year+1, 1, 1) - dt.timedelta(days=1)
    else:
        end = dt.date(year, m2+1, 1) - dt.timedelta(days=1)
    return start.isoformat(), end.isoformat()

def s1_min(aoi, start, end, pol):
    """Min-composite Sentinel-1 GRD over period to stabilize speckle."""
    return (ee.ImageCollection('COPERNICUS/S1_GRD')
            .filterBounds(aoi)
            .filterDate(start, end)
            .filter(ee.Filter.eq('instrumentMode','IW'))
            .filter(ee.Filter.listContains('transmitterReceiverPolarisation', pol))
            .select(pol)
            .min()
            .clip(aoi))

def classify_water(img_min, pol, threshold_db):
    """Water = backscatter < threshold (in dB); returns self-masked binary."""
    return img_min.lt(threshold_db).selfMask()

def area_km2(mask_img, aoi, scale=30, band_name='constant'):
    """Compute km² of a self-masked image."""
    area = (mask_img.multiply(ee.Image.pixelArea())
            .reduceRegion(ee.Reducer.sum(), aoi, scale, maxPixels=1e12))
    # selfMask() produces band 'constant'
    return ee.Number(area.get(band_name)).divide(1e6)

def landsat5_c2_sr_mask_scale(img):
    """L5 C2 L2 scaling + cloud/shadow masking."""
    qa = img.select('QA_PIXEL')
    cloud  = 1 << 3
    shadow = 1 << 4
    mask = qa.bitwiseAnd(cloud).eq(0).And(qa.bitwiseAnd(shadow).eq(0))
    optical = img.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal = img.select('ST_B6').multiply(0.00341802).add(149.0)
    return (img.addBands(optical, None, True)
               .addBands(thermal, None, True)
               .updateMask(mask))

def landsat5_median(aoi, start_date, end_date, months=None):
    """Median composite with optional wrapped-month filter ((11,12),(1,4))."""
    col = (ee.ImageCollection('LANDSAT/LT05/C02/T1_L2')
           .filterBounds(aoi)
           .filterDate(start_date, end_date)
           .map(landsat5_c2_sr_mask_scale))
    if months:
        # e.g., months=((11,12),(1,4)) to handle dry-season wrap
        f = ee.Filter.Or(
            ee.Filter.calendarRange(int(months[0][0]), int(months[0][1]), 'month'),
            ee.Filter.calendarRange(int(months[1][0]), int(months[1][1]), 'month')
        )
        col = col.filter(f)
    return col.median().clip(aoi)

def mndwi(img):  # SR_B2 (Green), SR_B5 (SWIR)
    return img.normalizedDifference(['SR_B2','SR_B5']).rename('MNDWI')

def water_mask_from_mndwi(img, threshold=0.0):
    return img.gt(threshold).selfMask()

def chirps_sum_mm(aoi, start, end):
    """Return AOI-mean of CHIRPS precipitation sum (mm) over [start,end]."""
    col = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
           .filterBounds(aoi).filterDate(start, end).select('precipitation'))
    total = col.sum().reduceRegion(ee.Reducer.mean(), aoi, 5000, maxPixels=1e12)
    return ee.Number(total.get('precipitation'))

# (Optional) quick summary print
print(f"AOI_DELTA bounds: {CFG['AOI_DELTA'].bounds().getInfo()['coordinates'][0][0]} …")
print(f"AOI_TONLESAP bounds: {CFG['AOI_TONLESAP'].bounds().getInfo()['coordinates'][0][0]} …")
print("Setup complete ✅")


In [ ]:
# === Cell 2: Folders, imports, shared params ===
import json, os
from pathlib import Path
import numpy as np
import pandas as pd

# plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# output folders
OUT = Path("outputs")
ASSETS = OUT / "web_assets"
ASSETS.mkdir(parents=True, exist_ok=True)

# time windows
FLOOD_M1, FLOOD_M2 = CFG["FLOOD_MONTHS"]   # (8,9)
DRY_M1, DRY_M2     = CFG["DROUGHT_MONTHS"] # (3,4)

EVENTS = {k: pd.to_datetime(v) for k, v in CFG["EVENTS"].items()}

print("Assets dir:", ASSETS.resolve())


In [ ]:
# === Cell 3: Build annual series (flood/drought/precip) for AOIs ===

def s1_annual_flood_series(aoi, years, m1=FLOOD_M1, m2=FLOOD_M2,
                           th_vv=CFG["TH_VV_DB"], th_vh=CFG["TH_VH_DB"]):
    rows = []
    for y in years:
        start, end = _daterange_of_year_months(y, m1, m2)
        # VV
        vv_min = s1_min(aoi, start, end, 'VV')
        vv_mask = classify_water(vv_min, 'VV', th_vv)
        area_vv = float(area_km2(vv_mask, aoi, 30).getInfo() or 0.0)
        # VH
        vh_min = s1_min(aoi, start, end, 'VH')
        vh_mask = classify_water(vh_min, 'VH', th_vh)
        area_vh = float(area_km2(vh_mask, aoi, 30).getInfo() or 0.0)
        # precip (Aug–Sep total)
        precip = float(chirps_sum_mm(aoi, start, end).getInfo() or 0.0)
        rows.append({"year": y, "flood_vv_km2": area_vv, "flood_vh_km2": area_vh,
                     "precip_wet_mm": precip})
    return pd.DataFrame(rows)

def s1_annual_dry_series(aoi, years, m1=DRY_M1, m2=DRY_M2, th_vh=CFG["TH_VH_DB"]):
    """건기 수면(VH)만—Note06에서와 동일 지표"""
    rows = []
    for y in years:
        start, end = _daterange_of_year_months(y, m1, m2)
        vh_min = s1_min(aoi, start, end, 'VH')
        vh_mask = classify_water(vh_min, 'VH', th_vh)
        area_vh = float(area_km2(vh_mask, aoi, 30).getInfo() or 0.0)
        precip = float(chirps_sum_mm(aoi, start, end).getInfo() or 0.0)
        rows.append({"year": y, "dry_vh_km2": area_vh, "precip_dry_mm": precip})
    return pd.DataFrame(rows)

def try_load_or_compute(aoi_name, builder_fn, fname):
    f = ASSETS / fname
    if f.exists():
        print(f"[LOAD] {fname}")
        return pd.read_csv(f)
    else:
        print(f"[BUILD] {fname}")
        if aoi_name == "Mekong_Delta":
            aoi = CFG["AOI_DELTA"]
        else:
            aoi = CFG["AOI_TONLESAP"]
        df = builder_fn(aoi, CFG["YEARS"])
        df.to_csv(f, index=False)
        return df

# ---- Load/build for both AOIs
df_delta_flood = try_load_or_compute("Mekong_Delta", s1_annual_flood_series, "annual_flood_delta.csv")
df_ts_flood    = try_load_or_compute("Tonle_Sap",    s1_annual_flood_series, "annual_flood_tonlesap.csv")

df_delta_dry = try_load_or_compute("Mekong_Delta", s1_annual_dry_series, "annual_dry_delta.csv")
df_ts_dry    = try_load_or_compute("Tonle_Sap",    s1_annual_dry_series, "annual_dry_tonlesap.csv")

# merge convenience
df_delta = pd.merge(df_delta_flood, df_delta_dry, on="year", how="outer").sort_values("year")
df_ts    = pd.merge(df_ts_flood,    df_ts_dry,    on="year", how="outer").sort_values("year")

display(df_delta.head(3), df_ts.head(3))


In [ ]:
# === Cell 4: Baselines and derived indicators ===

def landsat_baseline_wet_km2(aoi):
    # Note01과 동일 개념: 2005–2008 우기(5–10) MNDWI>0
    l5 = landsat5_median(aoi, "2005-01-01", "2008-12-31",
                         months=((5,10),(5,10)))
    mask = water_mask_from_mndwi(mndwi(l5), threshold=0.0)
    return float(area_km2(mask, aoi, 30).getInfo() or 0.0)

def landsat_baseline_dry_km2(aoi):
    # 2005–2008 건기(11–12 OR 1–4) MNDWI>0
    l5 = landsat5_median(aoi, "2005-01-01", "2008-12-31",
                         months=((11,12),(1,4)))
    mask = water_mask_from_mndwi(mndwi(l5), threshold=0.0)
    return float(area_km2(mask, aoi, 30).getInfo() or 0.0)

BASE = {}
for name, aoi in [("Mekong_Delta", CFG["AOI_DELTA"]), ("Tonle_Sap", CFG["AOI_TONLESAP"])]:
    print(f"[Baseline] computing wet/dry for {name} ...")
    BASE[name] = {
        "wet_km2": landsat_baseline_wet_km2(aoi),
        "dry_km2": landsat_baseline_dry_km2(aoi)
    }
    print(BASE[name])

# derived: VH gain vs VV (숨겨진 침수)
df_delta["vh_gain_km2"] = df_delta["flood_vh_km2"] - df_delta["flood_vv_km2"]
df_ts["vh_gain_km2"]    = df_ts["flood_vh_km2"]    - df_ts["flood_vv_km2"]


In [ ]:
# === Cell 5: Figure 1 — Annual flood extent (VV/VH) with baseline & events ===

def fig_annual_flood(df_delta, df_ts):
    fig = make_subplots(
        rows=1, cols=2, subplot_titles=("Mekong Delta — Flood (Aug–Sep)", "Tonlé Sap — Flood (Aug–Sep)")
    )
    # Delta
    fig.add_trace(go.Scatter(x=df_delta["year"], y=df_delta["flood_vv_km2"],
                             mode="lines+markers", name="Delta VV", legendgroup="D-VV"),
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=df_delta["year"], y=df_delta["flood_vh_km2"],
                             mode="lines+markers", name="Delta VH", legendgroup="D-VH"),
                  row=1, col=1)
    fig.add_hline(y=BASE["Mekong_Delta"]["wet_km2"], line_dash="dash", line_color="red",
                  annotation_text=f"Pre-dam wet baseline: {BASE['Mekong_Delta']['wet_km2']:,.0f} km²",
                  row=1, col=1)

    # Tonle Sap
    fig.add_trace(go.Scatter(x=df_ts["year"], y=df_ts["flood_vv_km2"],
                             mode="lines+markers", name="Tonle VV", legendgroup="T-VV"),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=df_ts["year"], y=df_ts["flood_vh_km2"],
                             mode="lines+markers", name="Tonle VH", legendgroup="T-VH"),
                  row=1, col=2)
    fig.add_hline(y=BASE["Tonle_Sap"]["wet_km2"], line_dash="dash", line_color="red",
                  annotation_text=f"Pre-dam wet baseline: {BASE['Tonle_Sap']['wet_km2']:,.0f} km²",
                  row=1, col=2)

    # Events
    for label, t in EVENTS.items():
        fig.add_vline(x=t.year, line_dash="dot", line_color="crimson", opacity=0.5, row=1, col=1)
        fig.add_vline(x=t.year, line_dash="dot", line_color="crimson", opacity=0.5, row=1, col=2)

    fig.update_layout(height=480, width=1100, title_text="Annual Flood Extent (VV vs VH) with Baselines & Events",
                      hovermode="x unified")
    fig.update_yaxes(title_text="km²", row=1, col=1); fig.update_yaxes(title_text="km²", row=1, col=2)
    fig.update_xaxes(dtick=1)
    return fig

fig1 = fig_annual_flood(df_delta, df_ts)
fig1.show()

# Export JSON
(fig1_json_path := ASSETS / "fig_annual_flood.json").write_text(fig1.to_json(), encoding="utf-8")
print("Saved ->", fig1_json_path)


In [ ]:
# === Cell 6: Figure 2 — Flood vs Precip (scatter with fit & events) ===

def fig_flood_vs_precip(df, aoi_label):
    # df: columns year, flood_vh_km2, precip_wet_mm
    sub = df[["year","flood_vh_km2","precip_wet_mm"]].dropna().copy()
    if len(sub) >= 2:
        r = np.corrcoef(sub["precip_wet_mm"], sub["flood_vh_km2"])[0,1]
    else:
        r = np.nan
    # Linear fit
    if len(sub) >= 2:
        coeff = np.polyfit(sub["precip_wet_mm"], sub["flood_vh_km2"], 1)
        xfit = np.linspace(sub["precip_wet_mm"].min(), sub["precip_wet_mm"].max(), 100)
        yfit = coeff[0]*xfit + coeff[1]
    else:
        xfit = np.array([])
        yfit = np.array([])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=sub["precip_wet_mm"], y=sub["flood_vh_km2"],
                             mode="markers+text",
                             text=sub["year"].astype(str),
                             textposition="top center",
                             name=f"{aoi_label} (r={r:.2f})"))
    if xfit.size:
        fig.add_trace(go.Scatter(x=xfit, y=yfit, mode="lines", name="Linear fit", opacity=0.6))
    fig.update_layout(
        title=f"{aoi_label}: Flood (VH, km²) vs Precip (Aug–Sep, mm)",
        xaxis_title="Precip (mm)", yaxis_title="Flood area (km²)",
        height=450, width=540
    )
    return fig

fig2_delta = fig_flood_vs_precip(df_delta, "Mekong Delta")
fig2_ts    = fig_flood_vs_precip(df_ts,    "Tonlé Sap")

# side-by-side container
fig2 = make_subplots(rows=1, cols=2, subplot_titles=("Mekong Delta", "Tonlé Sap"))
for tr in fig2_delta.data: fig2.add_trace(tr, row=1, col=1)
for tr in fig2_ts.data:    fig2.add_trace(tr, row=1, col=2)
fig2.update_layout(title="Flood–Precip Correlation (Aug–Sep)", height=480, width=1100)
fig2.show()

(fig2_json_path := ASSETS / "fig_flood_vs_precip.json").write_text(fig2.to_json(), encoding="utf-8")
print("Saved ->", fig2_json_path)


In [ ]:
# === Cell 7: Figure 3 — Dry-season water vs precip (bi-axis) with event line ===

def fig_dry_biaxis(df, aoi_label):
    fig = make_subplots(rows=1, cols=1, specs=[[{"secondary_y": True}]])
    fig.add_trace(go.Bar(x=df["year"], y=df["dry_vh_km2"], name="Dry-season water (km²)"), secondary_y=False)
    fig.add_trace(go.Scatter(x=df["year"], y=df["precip_dry_mm"], mode="lines+markers",
                             name="Dry-season precip (mm)"),
                  secondary_y=True)
    # baselines
    fig.add_hline(y=BASE[aoi_label.replace(" ", "_")]["dry_km2"], line_dash="dash", line_color="red",
                  annotation_text=f"Pre-dam dry baseline: {BASE[aoi_label.replace(' ','_')]['dry_km2']:,.0f} km²")
    # event(2019-07)
    for label, t in EVENTS.items():
        fig.add_vline(x=t.year, line_dash="dot", line_color="crimson", opacity=0.6)

    fig.update_layout(title=f"{aoi_label}: Dry-season Water vs Precip",
                      height=460, width=720, hovermode="x unified")
    fig.update_xaxes(dtick=1)
    fig.update_yaxes(title_text="Water (km²)", secondary_y=False)
    fig.update_yaxes(title_text="Precip (mm)",  secondary_y=True)
    return fig

fig3_delta = fig_dry_biaxis(df_delta, "Mekong_Delta")
fig3_ts    = fig_dry_biaxis(df_ts,    "Tonle_Sap")

# combined
fig3 = make_subplots(rows=1, cols=2, subplot_titles=("Mekong Delta", "Tonlé Sap"),
                     specs=[[{"secondary_y": True}, {"secondary_y": True}]])
for tr in fig3_delta.data: fig3.add_trace(tr, row=1, col=1, secondary_y="y2" in tr.yaxis)
for tr in fig3_ts.data:    fig3.add_trace(tr, row=1, col=2, secondary_y="y2" in tr.yaxis)
fig3.update_layout(title="Dry-season Water vs Precip (bi-axis)", height=480, width=1100)
fig3.show()

(fig3_json_path := ASSETS / "fig_dry_biaxis.json").write_text(fig3.to_json(), encoding="utf-8")
print("Saved ->", fig3_json_path)


In [ ]:
# === Cell 8 (Optional): Figure 4 — Hidden Flooded Vegetation (VH gain over VV) ===

def fig_vh_gain(df, aoi_label):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=df["year"], y=df["vh_gain_km2"], name="VH-only gain (km²)"))
    fig.update_layout(title=f"{aoi_label}: Additional inundation detected by VH (vs VV)",
                      xaxis_title="Year", yaxis_title="km²", height=420, width=600)
    fig.update_xaxes(dtick=1)
    return fig

fig4_delta = fig_vh_gain(df_delta, "Mekong Delta")
fig4_ts    = fig_vh_gain(df_ts,    "Tonlé Sap")

fig4 = make_subplots(rows=1, cols=2, subplot_titles=("Mekong Delta", "Tonlé Sap"))
for tr in fig4_delta.data: fig4.add_trace(tr, row=1, col=1)
for tr in fig4_ts.data:    fig4.add_trace(tr, row=1, col=2)
fig4.update_layout(title="VH-only additional flooded vegetation", height=460, width=1100)
fig4.show()

(fig4_json_path := ASSETS / "fig_vh_gain.json").write_text(fig4.to_json(), encoding="utf-8")
print("Saved ->", fig4_json_path)


In [ ]:
# === Cell 9: Build manifest for dashboard ===
manifest = {
    "project": "Mekong SAR Flood Insight",
    "version": "1.0",
    "generated_utc": pd.Timestamp.utcnow().isoformat(),
    "events": CFG["EVENTS"],
    "aoi": {
        "delta_bbox": [104.30, 8.50, 106.90, 10.90],
        "tonlesap_bbox": [103.30, 12.00, 105.20, 13.70]
    },
    "baselines": BASE,
    "figures": {
        "annual_flood": "fig_annual_flood.json",
        "flood_vs_precip": "fig_flood_vs_precip.json",
        "dry_biaxis": "fig_dry_biaxis.json",
        "vh_gain": "fig_vh_gain.json"  # optional
    },
    "notes": {
        "data_sources": [
            "COPERNICUS/S1_GRD (ESA, VV/VH)",
            "UCSB-CHG/CHIRPS/DAILY (precipitation)",
            "LANDSAT/LT05/C02/T1_L2 (USGS/NASA)",
            "USGS/NASADEM_HGT (DEM)"
        ],
        "method": "Multi-temporal SAR thresholding (VV<VV_th, VH<VH_th) + Landsat MNDWI baselines",
        "message": "VV misses flooded vegetation. VH recovers hidden inundation under crops/forest."
    }
}
(ASSETS / "manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
print("Saved ->", ASSETS / "manifest.json")


In [ ]:
# === Cell 10: Write a minimal Streamlit app template ===
app_code = r"""
import json
from pathlib import Path
import plotly.graph_objects as go
import streamlit as st

ASSETS = Path("outputs/web_assets")

st.set_page_config(page_title="Mekong SAR Flood Insight", layout="wide")
st.title("Mekong SAR Flood Insight — Interactive Dashboard")

# load manifest
mf = json.loads((ASSETS / "manifest.json").read_text(encoding="utf-8"))
st.caption(f"Generated at (UTC): {mf['generated_utc']}")

col1, col2 = st.columns([2,1])

with col1:
    st.subheader("Annual Flood Extent (VV vs VH)")
    fig1_json = json.loads((ASSETS / mf["figures"]["annual_flood"]).read_text(encoding="utf-8"))
    st.plotly_chart(go.Figure(fig1_json), use_container_width=True)
    st.markdown("> **Baseline**: Pre-dam (2005–2008) wet-season water extent from Landsat5 MNDWI.")

with col2:
    st.subheader("Flood vs Precip (Aug–Sep)")
    fig2_json = json.loads((ASSETS / mf["figures"]["flood_vs_precip"]).read_text(encoding="utf-8"))
    st.plotly_chart(go.Figure(fig2_json), use_container_width=True)

st.divider()

col3, col4 = st.columns(2)
with col3:
    st.subheader("Dry-season Water vs Precip")
    fig3_json = json.loads((ASSETS / mf["figures"]["dry_biaxis"]).read_text(encoding="utf-8"))
    st.plotly_chart(go.Figure(fig3_json), use_container_width=True)
with col4:
    st.subheader("Additional Inundation (VH over VV)")
    try:
        fig4_json = json.loads((ASSETS / mf["figures"]["vh_gain"]).read_text(encoding="utf-8"))
        st.plotly_chart(go.Figure(fig4_json), use_container_width=True)
    except Exception:
        st.info("VH gain figure not found (optional).")

st.divider()
st.markdown("**Data**: COPERNICUS/S1_GRD, CHIRPS, Landsat5 C2, NASADEM.  \n"
            "**Method**: SAR thresholding + multi-pol comparison + Landsat baselines.")
"""

(Path("app_streamlit.py")).write_text(app_code, encoding="utf-8")
print("Saved -> app_streamlit.py")


In [ ]:
# === Cell 11: List artifacts ===
for p in sorted(ASSETS.glob("*.json")):
    print(" -", p.name, f"({p.stat().st_size/1024:.1f} kB)")
print("To run the demo:\n  streamlit run app_streamlit.py")
